In [13]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML,1.5.0-preview2"
#r "nuget:Microsoft.ML.Mkl.Components,1.5.0-preview2"

//Install XPlot package
#r "nuget:XPlot.Plotly,2.0.0"

// Install data analysis package
#r "nuget:Microsoft.Data.Analysis,0.2.0"

Installed package Microsoft.ML.Mkl.Components version 1.5.0-preview2

Installed package Microsoft.Data.Analysis version 0.2.0

Installed package XPlot.Plotly version 2.0.0

Installed package Microsoft.ML version 1.5.0-preview2

In [14]:
using Microsoft.ML;
using Microsoft.ML.Data;
public class TemperaturePoint
{
    [LoadColumn(0)]
    public string Date;

    [LoadColumn(1)]
    public float MinTemp;

}

public class TemperatureParsed
{
    public float MinTemp;
    public int Month;
    public int Year;
    public int Day;
    public DateTime Date;
    public float DaysSinceStart;
}

In [15]:
MLContext mlContext = new MLContext(seed: 0);
IDataView trainDataView = mlContext.Data.LoadFromTextFile<TemperaturePoint>("daily-minimum-temperatures-in-me.csv", hasHeader: true, separatorChar: ',');

// extract minimum date for later processing
IEnumerable<string> dateColumn = trainDataView.GetColumn<string>("Date").ToList();
DateTime minDate = DateTime.Parse(dateColumn.Min());

int totalRows = dateColumn.Count();
display(minDate);
display(totalRows);

1981-01-01 00:00:00Z

3650

In [16]:
public static List<TemperaturePoint> Head(MLContext mlContext, IDataView dataView, int numberOfRows = 4)
{
    string msg = string.Format("DataView: Showing {0} rows with the columns", numberOfRows.ToString());
    display(msg);
          
    var rows = mlContext.Data.CreateEnumerable<TemperaturePoint>(dataView, reuseRowObject: false)
                    .Take(numberOfRows)
                    .ToList();
    
    return rows;
}

display(h4("Showing a few rows from training DataView:"));

var fewRows = Head(mlContext, trainDataView, 5);
display(fewRows);

Showing a few rows from training DataView:

DataView: Showing 5 rows with the columns

index,Date,MinTemp
0,1/1/1981,20.7
1,1/2/1981,17.9
2,1/3/1981,18.8
3,1/4/1981,14.6
4,1/5/1981,15.8


In [17]:
using System;
using System.Collections.Generic;

Action<TemperaturePoint, TemperatureParsed> mapping = (input, output) =>
    {
        const string DATETIME_FORMAT = "MM/dd/yyyy";
        output.MinTemp = input.MinTemp;
        
        DateTime result = DateTime.Parse(input.Date);
        output.Day = result.Day;
        output.Month = result.Month;
        output.Year = result.Year;
        output.Date = result;
        output.DaysSinceStart = (result-minDate).Days;


    };

var estimator = mlContext.Transforms.CustomMapping(mapping, null)
                .Append(mlContext.Transforms.Concatenate(outputColumnName: "DaysSince",
                                                         inputColumnNames: new[] { "DaysSinceStart" }))
                .AppendCacheCheckpoint(mlContext);

public static List<TemperatureParsed> Head(MLContext mlContext, IDataView dataView, int numberOfRows = 4)
{
    string msg = string.Format("DataView: Showing {0} rows with the columns", numberOfRows.ToString());
    display(msg);
          
    var rows = mlContext.Data.CreateEnumerable<TemperatureParsed>(dataView, reuseRowObject: false)
                    .Take(numberOfRows)
                    .ToList();
    
    return rows;
}

var model = estimator.Fit(trainDataView);
var transformedData = model.Transform(trainDataView);
display(Head(mlContext, transformedData, 5));

DataView: Showing 5 rows with the columns

index,MinTemp,Month,Year,Day,Date,DaysSinceStart
0,20.7,1,1981,1,1981-01-01 00:00:00Z,0
1,17.9,1,1981,2,1981-01-02 00:00:00Z,1
2,18.8,1,1981,3,1981-01-03 00:00:00Z,2
3,14.6,1,1981,4,1981-01-04 00:00:00Z,3
4,15.8,1,1981,5,1981-01-05 00:00:00Z,4


In [22]:
//Extract some data into arrays for plotting:
 
int numberOfRows = 730;
float[] temps = transformedData.GetColumn<float>("MinTemp").Take(numberOfRows).ToArray();
DateTime[] dates = transformedData.GetColumn<DateTime>("Date").Take(numberOfRows).ToArray();

In [23]:
int numTrain = (int) (0.8 * totalRows);
display(numTrain);
IDataView trainData = mlContext.Data.FilterRowsByColumn(transformedData, "DaysSinceStart", upperBound: numTrain);
IDataView testData = mlContext.Data.FilterRowsByColumn(transformedData, "DaysSinceStart", lowerBound: numTrain);

2920

In [24]:
using XPlot.Plotly;


var chart = Chart.Plot(
    new Graph.Scattergl()
    {
        x = dates,
        y = temps
    }
);

chart.Width = 600;
chart.Height = 600;
display(chart);

Notice that the data has a sinusoidal pattern, similar to a cosine function with a period of 365 days (1 year).
Let's try fitting a linear regression model.

A linear regression model has a formula of `y=mx+b`, where `x` (called the independent variable) represents time and `y` is the dependent variable.

In [26]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

var forecastingPipeline = mlContext.Regression.Trainers.Ols(
    labelColumnName: "MinTemp", 
    featureColumnName: "DaysSince");
    
var forecaster = forecastingPipeline.Fit(trainData);

// Use trained model to make inferences on test data
IDataView testDataPredictions = forecaster.Transform(testData);

// Extract model metrics and get RSquared
RegressionMetrics trainedModelMetrics = mlContext.Regression.Evaluate(testDataPredictions);
double rSquared = trainedModelMetrics.RSquared;
display(rSquared);

Unhandled exception: System.ArgumentOutOfRangeException: Label column 'Label' not found (Parameter 'schema')
   at Microsoft.ML.Data.RoleMappedSchema.MapFromNames(DataViewSchema schema, IEnumerable`1 roles, Boolean opt)
   at Microsoft.ML.Data.RoleMappedSchema..ctor(DataViewSchema schema, IEnumerable`1 roles, Boolean opt)
   at Microsoft.ML.Data.RoleMappedData..ctor(IDataView data, Boolean opt, KeyValuePair`2[] roles)
   at Microsoft.ML.Data.RegressionEvaluator.Evaluate(IDataView data, String label, String score)
   at Microsoft.ML.RegressionCatalog.Evaluate(IDataView data, String labelColumnName, String scoreColumnName)
   at Submission#28.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [12]:
var forecastingPipeline = mlContext.Forecasting.ForecastBySsa(
    outputColumnName: "ForecastedRentals",
    inputColumnName: "TotalRentals",
    windowSize: 7,
    seriesLength: 30,
    trainSize: 365,
    horizon: 7,
    confidenceLevel: 0.95f,
    confidenceLowerBoundColumn: "LowerBoundRentals",
    confidenceUpperBoundColumn: "UpperBoundRentals");

Unhandled exception: (1,49): error CS1061: 'ForecastingCatalog' does not contain a definition for 'ForecastBySsa' and no accessible extension method 'ForecastBySsa' accepting a first argument of type 'ForecastingCatalog' could be found (are you missing a using directive or an assembly reference?)

In [13]:
static void Evaluate(IDataView testData, ITransformer model, MLContext mlContext)
{
    IDataView predictions = model.Transform(testData);
    IEnumerable<float> actual =
    mlContext.Data.CreateEnumerable<TemperatureParsed>(testData, true)
        .Select(observed => observed.MinTemp);
    IEnumerable<float> forecast =
    mlContext.Data.CreateEnumerable<TemperatureParsed>(predictions, true)
        .Select(prediction => prediction.MinTemp);
}